In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [ ]:
queries = pd.read_csv('within-150m-time.csv')
train_queries = queries[queries['time'] <= 20110000]
paths = pd.read_fwf('paths(0:10996).csv', header=None,infer_nrows=10997)
paths = paths[0].str.split(',', expand=True)
paths.replace(to_replace=[None], value= -1, inplace=True)
lst = train_queries.index.values.tolist()
train_paths = paths[paths.index.isin(lst)]


In [ ]:
train_queries.reset_index(drop=True, inplace=True)
train_paths.reset_index(drop=True, inplace=True)

train_queries.to_csv('train_queries.csv',index=False)
train_paths.to_csv('train_paths.csv',index=False,header=False)


In [ ]:
#finding duplicate paths

paths = pd.read_fwf('train_paths.csv', header=None,infer_nrows=10997)
paths = paths[0].str.split(',', expand=True)
paths.replace(to_replace=[None], value= -1, inplace=True)
queries = pd.read_csv('train_queries.csv') 
queries=queries.rename(columns = {'Unnamed: 0':'path_no'})

paths = paths.apply(pd.to_numeric)
a = (paths.values)
len_path = []
for j,path in enumerate(a):
    l = np.count_nonzero(path+1)
    len_path.append(l)
    
profit_path = 9614*[0]

for i,path in enumerate(a):
    profit = 0
    if i>=0:          
        max_node = (max(path[0],path[len_path[i]-1]))
        min_node = (min(path[0],path[len_path[i]-1]))
        c = []
        for j,src_des in queries.iterrows():
                src = min(src_des[0],src_des[1])
                des = max(src_des[0],src_des[1])
                if (des==max_node and src==min_node):
                    c.append(j)
          
        profit  = min(c)
        profit_path[profit] += 1
        print(i,'of 0-all : ',profit)
        
                    
pd.DataFrame(profit_path).to_csv('duplicates.csv',index=False,header=False) 

In [ ]:
#computing profits
import pandas as pd
import numpy as np
paths = pd.read_fwf('train_paths.csv', header=None,infer_nrows=10997)
paths = paths[0].str.split(',', expand=True)
paths.replace(to_replace=[None], value= -1, inplace=True)
queries = pd.read_csv('train_queries.csv') 


paths = paths.apply(pd.to_numeric)
a = (paths.values)
a.sort(axis=1)
a = a[:, ::-1]
len_path = []
for j,path in enumerate(a):
    l = np.count_nonzero(path+1)
    len_path.append(l)
    
profit_path = []

for i,path in enumerate(a):
    profit = 0
    if i>=0:
        max_node = path[0]
        min_node = path[len_path[i]-1]
        for j,src_des in queries.iterrows():
            src = min(src_des[0],src_des[1])
            des = max(src_des[0],src_des[1])
            if des<=max_node and src>=min_node:
                c = 0
                for k in range(len_path[i]):
                    if src == path[k]:
                        c +=1
                    if des == path[k]:
                        c +=1
                if c == 2:
                    profit += 1

        profit_path.append(profit)
        print(i,'of 0-all profit : ',profit) 
                    
pd.DataFrame(profit_path).to_csv('profit.csv',index=False,header=False) 

In [ ]:
paths = pd.read_fwf('train_paths.csv', header=None,infer_nrows=4251)
paths = paths[0].str.split(',', expand=True)
paths.replace(to_replace=[None], value= -1, inplace=True)

profit = pd.read_csv('profit(0:9613).csv',header = None)

paths = paths.apply(pd.to_numeric)
a = (paths.values)
a.sort(axis=1)
a = a[:, ::-1]
len_path = []
for j,path in enumerate(a):
    l = np.count_nonzero(path+1)
    len_path.append(l)
    
ratio_path = []
for i,pro in profit.iterrows():
    ratio_path.append(pro[0]/len_path[i])
    
pd.DataFrame(ratio_path).to_csv('ratio(0:9613).csv',index=False,header=False)

pd.DataFrame(len_path).to_csv('length(0:9613).csv',index=False,header=False)

ratio = pd.read_csv('ratio(0:9613).csv',header = None)
profit = pd.read_csv('profit(0:9613).csv',header = None)
length = pd.read_csv('length(0:9613).csv',header = None)

paths = pd.read_fwf('train_paths.csv', header=None,infer_nrows=4251)
paths = paths[0].str.split(',', expand=True)
paths.replace(to_replace=[None], value= -1, inplace=True)
paths = paths.apply(pd.to_numeric)
a = (paths.values)
a.sort(axis=1)
a = a[:, ::-1]

frames = [ratio, profit, length]
merge = pd.concat(frames,ignore_index=True,axis = 1)

merge.to_csv('ratio-profit-length(0:9613).csv',index=False,header=False)

sorted_merge = merge.sort_values(by=[0], ascending=False)

sorted_merge.to_csv('sorted-ratio-profit-length(0:9613).csv',header=False)



In [ ]:
sort = pd.read_csv("sorted(0:9613).csv",header = None)
sort.columns=['path_number','number_of_queries_mapped']
sort = sort[0:5759]

paths = pd.read_csv('train_paths.csv', header=None)
a = (paths.values)
a.sort(axis=1)
a = a[:, ::-1]
len_path = []
for j,path in enumerate(a):
    l = np.count_nonzero(path+1)
    len_path.append(l)
    
queries = pd.read_csv('train_queries.csv') 

sorted_ratio = pd.read_csv('sorted-ratio-profit-length(0:9613).csv',header = None)
sorted_ratio.columns = ['path_no','ratio','profit','length']

lst = sort.path_number.values.tolist()
newdf = sorted_ratio[sorted_ratio.path_no.isin(lst)]
newdf=newdf.reset_index(drop=True)
newdf.to_csv('sorted-ratio-profit-length(0:5758).csv',header = False,index = False)

sort.to_csv('train_path_no_with_duplicate_paths_count(sorted).csv',header = None,index=None)

duplicate_count = pd.read_csv('train_path_no_with_duplicate_paths_count(sorted).csv',header = None)
duplicate_count.columns=['path_no','duplicate_count']

ratio = pd.read_csv('sorted-ratio-profit-length(0:5758).csv',header = None)
ratio.columns=['path_no','ratio','profit','length']

queries = pd.read_csv('train_queries.csv')

paths = pd.read_csv('train_paths.csv', header=None)
a = (paths.values)
a.sort(axis=1)
a = a[:, ::-1]
len_path = []
for j,path in enumerate(a):
    l = np.count_nonzero(path+1)
    len_path.append(l)
    
lst = ratio.path_no.values.tolist()
newdf = paths[paths.index.isin(lst)]
newdf.to_csv('train_paths_without_duplicates.csv')

a=pd.read_csv('train_paths_without_duplicates.csv')
a=a.rename(columns = {'Unnamed: 0':'path_no'})


In [ ]:
ratio = pd.read_csv('sorted-ratio-profit-length(0:5758).csv',header = None)
ratio.columns = ['path_no','ratio','profit','length']
def cachelist(cache,sorted_ratio,cumulative_length):
    capacity  = cache * cumulative_length
    b = 0
    a = 0
    cache_pathlist = []
    for i,data in sorted_ratio.iterrows():
        cache_pathlist.append(data[0])
        a += data[3]
        b += 1
        if a>=capacity:
            break
        
    return cache_pathlist

cache = 0.001
cumulative_length = 0
total_paths = 0
for i,data in ratio.iterrows():
    cumulative_length += data[3]
    total_paths += 1
for i in range(1000):
    cache_pathlist = cachelist(cache,ratio,cumulative_length)
    print(len(cache_pathlist)) 
    pd.DataFrame(cache_pathlist).to_csv('cached/cache_'+str("%0.3f"% cache)+'_pathlist.csv',index=False,header=False)
    cache +=0.001

In [ ]:
queries = pd.read_csv('within-150m-time.csv')
test_queries = queries[(queries['time'] >= 20110000) & (queries['time'] <= 20120000)]
test_queries.reset_index(drop=True, inplace=True)

paths = pd.read_csv('train_paths_without_duplicates.csv')
paths=paths.rename(columns = {'Unnamed: 0':'path_no'})

ans =[]
ans.append(0)
cache = 0
cache_id = 0
for z in tqdm(range(1000)):
    cache = cache+0.001
    cache_pathlist = pd.read_csv('cached/cache_'+str("%0.3f"% cache)+'_pathlist.csv',header = None)
    y = cache_pathlist.shape[0]
    cache_pathlist = cache_pathlist[cache_id:cache_pathlist.shape[0]]
    cache_id = y
    cache_list = cache_pathlist[0].values.tolist()
    p = paths[paths.path_no.isin(cache_list)]
    p = p.drop(['path_no'],axis =1)
    p = p.apply(pd.to_numeric)
    a = (p.values)
    a.sort(axis=1)
    a = a[:, ::-1]
    len_path = []
    for j,path in enumerate(a):
        l = np.count_nonzero(path+1)
        len_path.append(l)

    answer = 0
    indexes_to_drop=[]
    for i,src_des in test_queries.iterrows():
        des = max(src_des[0],src_des[1])
        src = min(src_des[0],src_des[1])
        for j,path in enumerate(a):
            max_node = path[0]
            min_node = path[len_path[j]-1]
            if des<=max_node and src>=min_node:
                c = 0
                for k in range(len_path[j]):
                    if src == path[k]:
                        c +=1
                    if des == path[k]:
                        c +=1
                if c == 2:
                    print(i)
                    answer +=1
                    indexes_to_drop.append(i)
                        
                    break
       
    ans.append(answer+ans[z])
    print(z,answer+ans[z])
    test_queries.drop(test_queries.index[indexes_to_drop],inplace = True)

    test_queries.reset_index(drop=True, inplace=True)

print(ans)
pd.DataFrame(ans[1:]).to_csv('cached/ans.csv',index = False,header = False)

In [ ]:
import matplotlib.pyplot as plt

cache =0.001*100
x = [cache*i for i in range(1,len(ans)+1)]
y = [(i/942)*100 for i in ans]

plt.plot(x,y)
plt.xlabel('Percent(%) of Cache stored')
plt.ylabel('Percent(%) of Queries answered')
plt.title('2007-2010 trained, tested on 2011')
plt.savefig('78910.eps',dpi = 600)